## Datarensning


In [4]:
# This will handle loading and cleaning the Excel data for brukare and medarbetare.

import pandas as pd

def load_data(file_path):
    """
    Load Excel data and return a dictionary of dataframes.
    """
    # Load the Excel file from the given file path
    excel_data = pd.ExcelFile(file_path)
    
    # Parse the sheets into separate dataframes
    data = {
        'brukare': excel_data.parse('Individer, brukare', header=1),
        'medarbetare': excel_data.parse('Medarbetare', header=2)
    }

    return data


def clean_data(brukare_df, medarbetare_df):
    """
    Clean and structure brukare and medarbetare data for analysis.
    """
    # Rename 'Unnamed: 0' to 'Individ' for brukare_df
    brukare_df.rename(columns={'Unnamed: 0': 'Individ'}, inplace=True)

    # Rename 'Unnamed: 0' to 'Medarbetare' for medarbetare_df
    medarbetare_df.rename(columns={'Unnamed: 0': 'Medarbetare'}, inplace=True)

    # Remove brukare with missing or invalid 'Individ' or 'Adress'
    brukare_df = brukare_df.dropna(subset=['Individ', 'Adress']).copy()
    brukare_df = brukare_df[brukare_df['Individ'] != '-']  # Remove rows where 'Individ' is '-'
    brukare_df = brukare_df[brukare_df['Adress'] != '-']  # Remove rows where 'Adress' is '-'
    
    # Fill missing values in other columns with placeholder '-'
    brukare_df.fillna('-', inplace=True)
    medarbetare_df.fillna('-', inplace=True)

    # Convert relevant columns to boolean
    brukare_df['Kräver körkort'] = brukare_df['Kräver körkort'].apply(lambda x: x == 'Ja')
    medarbetare_df['Körkort'] = medarbetare_df['Körkort'].apply(lambda x: x == 'Ja')

    return brukare_df, medarbetare_df


data = load_data("Studentuppgift fiktiv planering.xlsx")

brukare_df, medarbetare_df = clean_data(data["brukare"], data["medarbetare"])

brukare_df
medarbetare_df

,Medarbetare,Tål hund,Tål katt,Tål rök,Man,Kvinna,Körkort,Läkemedelsdelegering,Insulindelegering,Stomidelegering,18 år el mer
0,Medarbetare 1,Ja,Ja,Ja,Nej,Ja,True,Nej,Nej,Nej,Ja
1,Medarbetare 2,Ja,Ja,Ja,Ja,Nej,False,Nej,Nej,Nej,Nej
2,Medarbetare 3,Ja,Ja,Ja,Ja,Nej,True,Ja,Ja,Ja,Ja
3,Medarbetare 4,Ja,Ja,Ja,Nej,Ja,True,Ja,Ja,Ja,Ja
4,Medarbetare 5,Ja,Ja,Ja,Nej,Ja,True,Ja,Ja,Ja,Ja
5,Medarbetare 6,Ja,Ja,Ja,Ja,Nej,True,Ja,Nej,Nej,Ja
6,Medarbetare 7,Ja,Ja,Ja,Nej,Ja,True,Ja,Ja,Ja,Ja
7,Medarbetare 8,Ja,Ja,Ja,Nej,Ja,True,Ja,Ja,Ja,Ja
8,Medarbetare 9,Ja,Ja,Ja,Ja,Nej,True,Ja,Ja,Ja,Ja
9,Medarbetare 10,Ja,Ja,Nej,Nej,Ja,True,Ja,Ja,Ja,Ja
